In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
#pulling data API
key =credentials.login['key_isaham']  # extract password

In [2]:
# Define the instruments to download.
#tech = ["GREATEC", 
       #"INARI"]

tech = ["GREATEC",
        "UWC",       
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ]   
       
# looping through all stocks
# dictionary of dataframes
ss={}
d = {}
# listing all stocks
all_stocks = [] 
sp={}
nsp={}


for m,stock in enumerate (tech):
        
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=False)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 
    
    
    df['Stock_Name'] = symbol

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    #print(oversold[oversold == True])
    #print(df)
    # scaling certain features only
    col_names_pred = ['c', 'l', 'o', 'v','h']  
    features_pred = df[col_names_pred]
    
    #features = dataf[col_names]
    scaler_pred = StandardScaler().fit(features_pred.values)
    features_pred = scaler_pred.transform(features_pred.values)
    # convert the array back to a dataframe
    dataset_pred = DataFrame(features_pred)
    
    
    col_notscale_pred=['d','Stock_Name','Buy OS']
    features_notscale_pred=df[col_notscale_pred]
    
    # saving in multiple df
    sp["{}".format(stock)] = dataset_pred
    nsp["{}".format(stock)] = features_notscale_pred
    
    #mk["{}".format(stock)] = df
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []


    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
        #print("Position closed")
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #Signal_buy = []
    #for t in range(len(trans)):
        #Signal_buy.append(1)
        
    
    trans['Stock'] = pd.Series(Name_stock)
    #trans['Signal'] = pd.Series(Signal_buy)
    
    # scaling certain features only
    col_names = ['Buy', 'Low', 'Open', 'Volume','high']  
    features = trans[col_names]
    
    #features = dataf[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    # convert the array back to a dataframe
    dataset = DataFrame(features)
    
    
    col_notscale=['Buy_Date','Stock','wr']
    features_notscale=trans[col_notscale]
    
    # saving in multiple df
    d["{}".format(stock)] = dataset
    ss["{}".format(stock)] = features_notscale
    
    # list of all stocks
    all_stocks.append(d[stock])
    
    
    #print(trans)
    
    #dct_1={ 'Number of stock':dct
    
    #}
    #trans_1 = pd.DataFrame(dct_1)
    
    #print(trans[0:])
    
    #trans.to_csv('result {}.csv'.format(symbol), index=False)
    #C:\Users\Mo Syamil\Documents\data scientiest trial\strategy stho\result
            
    print("----------------DONE {}------------------".format(symbol))
    #print(trans)
    #concat_all= pd.concat([trans], axis=0)
    #print(concat_all)
    #data_1=[]
    
    #for d in  range(len(trans)):
        #concat_all=pd.concat([trans], axis=0)
        #data_1.append(list(trans))
        #print(data_1)
    #c_data= trans[m]
    #print(concat_all)
    #print(trans)
    
    
        
    #print(trans)
    #print()

    
# Merging all df
#concat_all= pd.concat([trans], axis=0)
#print(trans.append(m))
#print(trans)
#print(concat_all)

#print("Finish")

    



----------------PROCESSING GREATEC------------------
----------------DONE GREATEC------------------
----------------PROCESSING UWC------------------
----------------DONE UWC------------------
----------------PROCESSING D&O------------------
----------------DONE D&O------------------
----------------PROCESSING GENETEC------------------
----------------DONE GENETEC------------------
----------------PROCESSING UNISEM------------------
----------------DONE UNISEM------------------
----------------PROCESSING FRONTKN------------------
----------------DONE FRONTKN------------------
----------------PROCESSING ATAIMS------------------
----------------DONE ATAIMS------------------
----------------PROCESSING FPI------------------
----------------DONE FPI------------------
----------------PROCESSING PENTA------------------
----------------DONE PENTA------------------
----------------PROCESSING DUFU------------------
----------------DONE DUFU------------------
----------------PROCESSING VS---------

In [3]:
# combining all data generated
df_notscale_pred = pd.concat(list(nsp.values()),ignore_index=True)

In [4]:
df_notscale_pred

,d,Stock_Name,Buy OS
0,2019-06-13,GREATEC,False
1,2019-06-14,GREATEC,False
2,2019-06-17,GREATEC,False
3,2019-06-18,GREATEC,False
4,2019-06-19,GREATEC,False
...,...,...,...
58652,2021-04-01,VITROX,False
58653,2021-04-02,VITROX,False
58654,2021-04-05,VITROX,False
58655,2021-04-06,VITROX,False


In [5]:
df_scale_pred = pd.concat(list(sp.values()),ignore_index=True)

In [6]:
df_scale_pred=df_scale_pred.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})

In [7]:
df_scale_pred

,Buy,Low,Open,Volume,High
0,-1.156293,-1.175626,-1.181641,13.912507,-1.151553
1,-1.136561,-1.141149,-1.147851,7.940175,-1.130866
2,-1.111190,-1.119601,-1.125325,5.410807,-1.118453
3,-1.085820,-1.102363,-1.099982,6.723034,-1.090870
4,-1.092867,-1.086561,-1.080271,3.796542,-1.088112
...,...,...,...,...,...
58652,3.602816,3.476808,3.419179,0.053523,3.577322
58653,3.772348,3.698796,3.638019,-0.019175,3.766083
58654,3.772348,3.759843,3.774794,-0.139519,3.766083
58655,3.608285,3.648849,3.829504,-0.122576,3.809228


In [8]:
#df_scale=df_scale[['Buy', 'Low', 'Open', 'Volume','High']]

In [9]:
result_scaler_pred = pd.concat([df_notscale_pred, df_scale_pred], axis=1)

In [10]:
result_scaler_pred=result_scaler_pred.rename(columns={'d': 'Date'})

In [11]:
result_scaler_pred

,Date,Stock_Name,Buy OS,Buy,Low,Open,Volume,High
0,2019-06-13,GREATEC,False,-1.156293,-1.175626,-1.181641,13.912507,-1.151553
1,2019-06-14,GREATEC,False,-1.136561,-1.141149,-1.147851,7.940175,-1.130866
2,2019-06-17,GREATEC,False,-1.111190,-1.119601,-1.125325,5.410807,-1.118453
3,2019-06-18,GREATEC,False,-1.085820,-1.102363,-1.099982,6.723034,-1.090870
4,2019-06-19,GREATEC,False,-1.092867,-1.086561,-1.080271,3.796542,-1.088112
...,...,...,...,...,...,...,...,...
58652,2021-04-01,VITROX,False,3.602816,3.476808,3.419179,0.053523,3.577322
58653,2021-04-02,VITROX,False,3.772348,3.698796,3.638019,-0.019175,3.766083
58654,2021-04-05,VITROX,False,3.772348,3.759843,3.774794,-0.139519,3.766083
58655,2021-04-06,VITROX,False,3.608285,3.648849,3.829504,-0.122576,3.809228


In [12]:
df_all_pred = result_scaler_pred.sort_values(by="Date")
# setting Dates as index
df_all_pred.set_index('Date', inplace=True)

In [13]:
# Encoding the stock names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all_pred["Stock"] = le.fit_transform(df_all_pred["Stock_Name"])
df_all_pred

,Stock_Name,Buy OS,Buy,Low,Open,Volume,High,Stock
Date,,,,,,,,
1994-01-03,ATAIMS,False,1.701955,1.753783,1.709914,-0.254617,1.640919,0
1994-01-03,MPI,False,-0.535210,-0.526229,-0.470370,-0.596527,-0.479810,7
1994-01-04,ATAIMS,False,1.622721,1.637190,1.676119,-0.280536,1.624524,0
1994-01-04,MPI,False,-0.513892,-0.518969,-0.498829,-0.679078,-0.507671,7
1994-01-05,ATAIMS,False,1.651018,1.689657,1.642324,-0.194353,1.624524,0
...,...,...,...,...,...,...,...,...
2021-04-07,GENETEC,False,4.620094,4.639185,4.769930,-0.095452,4.592826,5
2021-04-07,D&O,False,9.131826,8.971849,8.819311,1.160330,8.939296,1
2021-04-07,UWC,False,2.095609,2.032100,2.076284,-0.335619,2.109481,11


### training data

In [14]:
# combining all data generated
df_notscale = pd.concat(list(ss.values()),ignore_index=True)

In [15]:
df_notscale

,Buy_Date,Stock,wr
0,2019-11-25,GREATEC,1
1,2019-11-27,GREATEC,1
2,2020-03-10,GREATEC,0
3,2020-09-11,GREATEC,1
4,2020-09-18,GREATEC,1
...,...,...,...
1340,2020-05-21,VITROX,1
1341,2020-09-11,VITROX,1
1342,2020-11-27,VITROX,1
1343,2021-02-24,VITROX,0


In [16]:
df_scale = pd.concat(list(d.values()),ignore_index=True)

In [17]:
df_scale=df_scale.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})

In [18]:
df_scale

,Buy,Low,Open,Volume,High
0,-1.422847,-1.398726,-1.394648,-0.218389,-1.426193
1,-1.437665,-1.416967,-1.397626,0.867358,-1.423269
2,-1.102767,-1.176798,-1.177276,2.146298,-1.098748
3,-0.213656,-0.261726,-0.239299,0.782518,-0.215816
4,-0.169201,-0.155322,-0.179745,-1.236101,-0.192427
...,...,...,...,...,...
1340,1.418559,1.449847,1.438214,-0.257703,1.449488
1341,2.533699,2.438409,2.432330,0.540154,2.500931
1342,3.224771,3.202541,3.181903,-0.426413,3.262321
1343,3.826842,3.827740,3.846419,-0.301918,3.780274


In [19]:
result_scaler = pd.concat([df_notscale, df_scale], axis=1)

In [20]:
result_scaler

,Buy_Date,Stock,wr,Buy,Low,Open,Volume,High
0,2019-11-25,GREATEC,1,-1.422847,-1.398726,-1.394648,-0.218389,-1.426193
1,2019-11-27,GREATEC,1,-1.437665,-1.416967,-1.397626,0.867358,-1.423269
2,2020-03-10,GREATEC,0,-1.102767,-1.176798,-1.177276,2.146298,-1.098748
3,2020-09-11,GREATEC,1,-0.213656,-0.261726,-0.239299,0.782518,-0.215816
4,2020-09-18,GREATEC,1,-0.169201,-0.155322,-0.179745,-1.236101,-0.192427
...,...,...,...,...,...,...,...,...
1340,2020-05-21,VITROX,1,1.418559,1.449847,1.438214,-0.257703,1.449488
1341,2020-09-11,VITROX,1,2.533699,2.438409,2.432330,0.540154,2.500931
1342,2020-11-27,VITROX,1,3.224771,3.202541,3.181903,-0.426413,3.262321
1343,2021-02-24,VITROX,0,3.826842,3.827740,3.846419,-0.301918,3.780274


In [21]:
df_all = result_scaler.sort_values(by="Buy_Date")
# setting Dates as index
df_all.set_index('Buy_Date', inplace=True)

In [22]:
# Encoding the stock names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all["Stock"] = le.fit_transform(df_all["Stock"])
df_all

,Stock,wr,Buy,Low,Open,Volume,High
Buy_Date,,,,,,,
1994-09-07,7,0,-0.752361,-0.758173,-0.755265,1.202233,-0.733711
1994-09-27,7,0,-0.733371,-0.746831,-0.746833,0.809197,-0.714979
1994-09-30,7,0,-0.722520,-0.755337,-0.729968,-0.396468,-0.723007
1994-09-30,0,0,0.649960,0.685527,0.672256,-0.329433,0.656679
1994-10-04,0,0,0.658738,0.676506,0.663335,-0.470362,0.635251
...,...,...,...,...,...,...,...
2021-03-04,6,1,1.351179,1.376893,1.428215,0.936792,1.389249
2021-03-09,0,1,0.711406,0.617868,0.636572,5.593638,0.686678
2021-03-10,7,1,3.447116,3.503234,3.725253,4.331648,3.561174


In [23]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import calendar
import datetime as dt

In [24]:
#defining the splitting date (for training we use 2 years data)
# splitting train test
day = dt.datetime.today().day
month = dt.datetime.today().month
year = dt.datetime.today().year - 2
date1 = "{}-{}-01".format(year, str(month).zfill(2), str(day).zfill(2))
date2 = str(dt.datetime.today().date() - dt.timedelta(days=1))  # until yesterday only

In [25]:
# Separate input features and target
#X = df_all.drop(['wr','Pct_change','Sell_Date','Sell','Volume','Low','high'], axis=1)
X = df_all.drop(['wr'], axis=1)
y = df_all.wr

In [26]:
# splitting train test
X_train, y_train = X[date1: date2], y[date1: date2]

In [27]:
y_train.value_counts()

1    129
0     28
Name: wr, dtype: int64

In [28]:
X_train

,Stock,Buy,Low,Open,Volume,High
Buy_Date,,,,,,
2019-04-01,0,0.259338,0.275061,0.266353,0.571245,0.249544
2019-04-04,12,1.182966,1.198699,1.188356,2.438212,1.174974
2019-04-11,12,1.185584,1.206715,1.209620,-0.563506,1.167204
2019-04-18,12,1.188202,1.206715,1.196330,-0.592901,1.172384
2019-05-03,9,1.448589,1.423960,1.476281,-0.291155,1.435670
...,...,...,...,...,...,...
2021-03-04,6,1.351179,1.376893,1.428215,0.936792,1.389249
2021-03-09,0,0.711406,0.617868,0.636572,5.593638,0.686678
2021-03-10,7,3.447116,3.503234,3.725253,4.331648,3.561174


In [29]:
y_train

Buy_Date
2019-04-01    1
2019-04-04    1
2019-04-11    1
2019-04-18    1
2019-05-03    0
             ..
2021-03-04    1
2021-03-09    1
2021-03-10    1
2021-03-12    1
2021-03-16    1
Name: wr, Length: 157, dtype: int64

In [30]:
#resampling

In [31]:
# concatenate our training data back together
X_concat = pd.concat([X_train, y_train], axis=1)

In [32]:
# separate minority and majority classes
not_win = X_concat[X_concat.wr==0]
win = X_concat[X_concat.wr==1]

In [33]:
# upsample minority
not_win_upsampled = resample(not_win,
                          replace=True, # sample with replacement
                          n_samples=len(win), # match number in majority class
                          random_state=0) # reproducible results

In [34]:
# combine majority and upsampled minority
upsampled = pd.concat([win, not_win_upsampled]).sort_values(by="Buy_Date")

In [35]:
upsampled.wr.value_counts()

1    129
0    129
Name: wr, dtype: int64

In [36]:
upsampled

,Stock,Buy,Low,Open,Volume,High,wr
Buy_Date,,,,,,,
2019-04-01,0,0.259338,0.275061,0.266353,0.571245,0.249544,1
2019-04-04,12,1.182966,1.198699,1.188356,2.438212,1.174974,1
2019-04-11,12,1.185584,1.206715,1.209620,-0.563506,1.167204,1
2019-04-18,12,1.188202,1.206715,1.196330,-0.592901,1.172384,1
2019-05-03,9,1.448589,1.423960,1.476281,-0.291155,1.435670,0
...,...,...,...,...,...,...,...
2021-03-04,6,1.351179,1.376893,1.428215,0.936792,1.389249,1
2021-03-09,0,0.711406,0.617868,0.636572,5.593638,0.686678,1
2021-03-10,7,3.447116,3.503234,3.725253,4.331648,3.561174,1


In [37]:
# trying logistic regression again with the balanced dataset
X_train = upsampled.drop('wr', axis=1)
y_train = upsampled.wr

# training the model
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [38]:
# taking today and yesterday data only
latest = df_all_pred[date2:]

In [39]:
latest

,Stock_Name,Buy OS,Buy,Low,Open,Volume,High,Stock
Date,,,,,,,,
2021-04-06,SKPRES,False,2.716796,2.782585,2.846179,0.161262,2.827468,9
2021-04-06,DUFU,False,3.992031,4.088037,4.184423,-0.279253,4.113301,2
2021-04-06,VITROX,False,3.608285,3.648849,3.829504,-0.122576,3.809228,12
2021-04-06,GREATEC,False,2.000924,1.986208,1.973497,-0.107238,1.970875,6
2021-04-06,ATAIMS,False,1.119017,1.164988,1.208628,1.573753,1.176407,0
2021-04-06,PENTA,False,4.536312,4.600844,4.506026,-0.280540,4.507433,8
2021-04-06,FRONTKN,False,4.318216,4.425041,4.524660,-0.355264,4.453360,4
2021-04-06,MPI,False,4.298423,4.316502,4.239497,-0.138547,4.253745,7
2021-04-06,FPI,False,4.217519,4.323711,4.325870,0.861093,4.215296,3


In [40]:
dct_m = {}
symbol =[]
predicted=[]
date=[]

for m in range(len(latest)):
    if latest["Buy OS"].iloc[m] == True:
        # predicting the train and test set
        inputs = latest.drop(['Stock_Name','Buy OS'] ,axis=1).iloc[m].values
        
        #print(inputs)
        inputs = inputs.reshape(1,-1)
        y_pred = upsampled.predict(inputs)
        pred1 = [round(value) for value in y_pred]
        #print(pred1)
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        
        dct_m["predicted_class"] =  pred1[0]
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = dt.datetime.today()
        date.append(dct_m['date_uploaded'])
        print(dct_m)
    else:
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        dct_m["predicted_class"] = "No buy signal. No prediction"
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = dt.datetime.today()
        date.append(dct_m['date_uploaded'])
        #print("None of the stock in WL shows buy signal.\n No prediction")
        pass
        print(dct_m)
    mongo_db = {
        'Symbol' :symbol,
        "Predicted_Class" :predicted,
        'Date_uploaded':date
    }
    Mongo_db = pd.DataFrame(mongo_db) 
    

{'symbol': 'SKPRES', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 790313)}
{'symbol': 'DUFU', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 790313)}
{'symbol': 'VITROX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 791313)}
{'symbol': 'GREATEC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 791313)}
{'symbol': 'ATAIMS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 792314)}
{'symbol': 'PENTA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 792314)}
{'symbol': 'FRONTKN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': datetime.datetime(2021, 4, 7, 17, 19, 21, 793313)}
{'symbol': 'MPI', 'pr

In [41]:
Mongo_db

,Symbol,Predicted_Class,Date_uploaded
0,SKPRES,No buy signal. No prediction,2021-04-07 17:19:21.790313
1,DUFU,No buy signal. No prediction,2021-04-07 17:19:21.790313
2,VITROX,No buy signal. No prediction,2021-04-07 17:19:21.791313
3,GREATEC,No buy signal. No prediction,2021-04-07 17:19:21.791313
4,ATAIMS,No buy signal. No prediction,2021-04-07 17:19:21.792314
5,PENTA,No buy signal. No prediction,2021-04-07 17:19:21.792314
6,FRONTKN,No buy signal. No prediction,2021-04-07 17:19:21.793313
7,MPI,No buy signal. No prediction,2021-04-07 17:19:21.794317
8,FPI,No buy signal. No prediction,2021-04-07 17:19:21.795314
9,GENETEC,No buy signal. No prediction,2021-04-07 17:19:21.795314
